In [3]:
def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

# Example usage
input_string = 'AGATAAGG'
sorted_substrings = generate_sorted_substrings(input_string)

sorted_substrings

['AGATAAGG',
 'AGATAAG',
 'GATAAGG',
 'AGATAA',
 'ATAAGG',
 'GATAAG',
 'AGATA',
 'ATAAG',
 'GATAA',
 'TAAGG',
 'AAGG',
 'AGAT',
 'ATAA',
 'GATA',
 'TAAG',
 'AAG',
 'AGA',
 'AGG',
 'ATA',
 'GAT',
 'TAA',
 'AA',
 'AG',
 'AT',
 'GA',
 'GG',
 'TA',
 'A',
 'G',
 'T']

In [ ]:
import pandas as pd

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.',''))
    return sequences

def ref_seq(sequences):
    for sequence in sequences:
        return sequence
    return None

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

def align_sequences(sequences, reference_sequence):
    ref_index = 0
    substrings = generate_sorted_substrings(reference_sequence)

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if reference_sequence in sequence:
            idx = sequence.find(reference_sequence)
            aligned = True
            entire = True
        else:
            for substring in substrings:
                if substring in sequence:
                    idx = sequence.find(substring)
                    ref_index2 = reference_sequence.find(substring)
                    entire = False
                    aligned = True
                    break

        if aligned:
            if entire:
                offset = ref_index - idx
            else:
                offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
        
    return df

for i in ["Cebpa","Cebpb"]:
    input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/GATA4/GATA4_ SMALL_testing.txt'
    output_file_path = 'new_algo_GATA_aligned_sequences.txt'

    sequences = read_sequences_from_file(input_file_path)
    
    reference_sequence = sequences[0]
    if not reference_sequence:
        raise ValueError(f"No sequence containing '{reference_sequence}' was found in the file.")

    df_aligned = align_sequences(sequences, reference_sequence)
    
    columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
    extracted_data = df_aligned[columns_to_extract]
    extracted_data = extracted_data.fillna('-')

    formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

    with open(output_file_path, 'w') as file:
        for line in formatted_data:
            file.write(line + '\n')

In [9]:
df_aligned

,-7,-6,-5,-4,-3,-2,-1,0,1,2,...,7,8,9,10,11,12,13,14,15,16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,...,A,G,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,A,C,T,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,...,A,G,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,G,...,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,C,...,A,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,T,C,C,C,C,C,T,A
58062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,T,A,NaN,NaN,NaN,NaN,NaN,NaN
58063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,A,T,C,T,A,T,C,C,NaN
58064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN


# Generating txt file for weblogo

In [ ]:
columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
extracted_data = df_aligned[columns_to_extract]
extracted_data = extracted_data.fillna('-')

formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

output_file_path = 'Elf2_whole_aligned.txt'
with open(output_file_path, 'w') as file:
    for line in formatted_data:
        file.write(line + '\n')

# Better Algo

In [5]:
import pandas as pd

def read_sequences_from_file(file_path):
    with open(file_path, 'r') as file:
        next(file)
        sequences = []
        for line in file:
            columns = line.split()
            sequences.append(columns[0].replace('.',''))
    return sequences

def ref_seq(sequences):
    for sequence in sequences:
        return sequence
    return None

def generate_sorted_substrings(s):
    # Generate all possible substrings
    substrings = [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]
    
    # Sort the substrings first by reverse length, then alphabetically
    substrings_sorted = sorted(substrings, key=lambda x: (-len(x), x))
    
    # Remove duplicates without using a set
    unique_substrings = []
    for substring in substrings_sorted:
        if substring not in unique_substrings:
            unique_substrings.append(substring)
    
    return unique_substrings

def align_sequences(sequences, reference_sequence):
    ref_index = 0
    substrings = generate_sorted_substrings(reference_sequence)

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if reference_sequence in sequence:
            idx = sequence.find(reference_sequence)
            aligned = True
            entire = True
        else:
            prev_best_number_of_matches = 0
            total_number_of_matches = 0
            best_substring_index = -1
        
            for i in range(0, len(substrings)):
                if substrings[i] in sequence:
                    for j in range(i-1, len(substrings)):
                        if substrings[j] in sequence:
                            total_number_of_matches += 1
                    
                    if prev_best_number_of_matches < total_number_of_matches:
                        best_substring_index = i
                    
            
            best_substring = substrings[best_substring_index]
            idx = sequence.find(best_substring)
            ref_index2 = reference_sequence.find(best_substring)
            entire = False
            aligned = True
                    

        if aligned:
            if entire:
                offset = ref_index - idx
            else:
                offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = pd.concat([df, pd.DataFrame([list(sequence)], columns=indices)], ignore_index=True)

        
    return df
#"Cebpa","Cebpb"
for i in ['ETV-5']:
    input_file_path = f'/Users/bristi/Desktop/Design Project/Working-with-TF/{i}/{i}_8mers_top_enrichment.txt'
    output_file_path = f'new_algo_whole_{i}_aligned_sequences.txt'

    sequences = read_sequences_from_file(input_file_path)

    reference_sequence = sequences[0]
    if not reference_sequence:
        raise ValueError(f"No sequence containing '{reference_sequence}' was found in the file.")

    df_aligned = align_sequences(sequences, reference_sequence)

    columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
    extracted_data = df_aligned[columns_to_extract]
    extracted_data = extracted_data.fillna('-')

    formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

    with open(output_file_path, 'w') as file:
        for line in formatted_data:
            file.write(line + '\n')